# Packages 

In [70]:
%load_ext autoreload
%autoreload 2
import sys

import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *
from src.case_analysis import show_single_case
# try github 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

In [71]:
debug = False

task = 'task2'

version = 'v2'
# # target locales: locales needed for task1
if task == 'task1':
    target_locals = ["DE", 'JP', 'UK']
elif task == 'task2':
    target_locals = ['ES', 'FR', 'IT']
else:
    assert 1 == 2


ranker_train_data_dir = f'../data/rank_train_data_{task}_{version}'

rank_model_version = f'{task}_rank_lgbm_{version}'

In [72]:
rank_model_version

'task2_rank_lgbm_v2'

In [73]:
! ls ../data/

candidates	 rank_train_data_task2	   raw_data		  sub_files
eval_data	 rank_train_data_task2_v2  raw_data_session_id	  train_data
rank_train_data  rank_train_data_v2	   session_item_features


In [74]:

if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'


w2v_model_version = f'w2v_{task}'
nic_model_version = f'nic_{task}'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
# w2v_topn=100
# nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")


sub_file = f'../data/sub_files/{rank_model_version}.parque'
eval_sub_file = f'../data/sub_files/eval_{rank_model_version}.parque'

In [75]:
rank_model_dir

'../model_training/task2_rank_lgbm_v2'

In [76]:
! mkdir {rank_model_dir}

mkdir: cannot create directory ‘../model_training/task2_rank_lgbm_v2’: File exists


In [77]:
sub_file

'../data/sub_files/task2_rank_lgbm_v2.parque'

In [78]:
eval_sub_file

'../data/sub_files/eval_task2_rank_lgbm_v2.parque'

## Ranker config

In [79]:
estimator = 1000
early_stop_rounds = 10
eval_frac = 0.1
if debug:
    early_stop_rounds = 5
    
target = 'target'
feature_cols = [
    'next_item_weight'
    , 'next_item_weight_nfi'
    , 'next_item_weight_co_visit'
    , 'locale'
    , 'w2v_weight'
    , 'last_item_similarity'
    # , 'prev_item_similarity'	
    , 'prev_length'
    , 'price_mean_norm'	
    , 'price_last_norm'
    , 'price_min_norm'
    , 'price_max_norm'
    
    
    ,'next_item_cnt', 
    'next_item_normalized_weight',
    'discount_weight', 'following_weight', 
    'following_discount_weight', 'previous_weight', 
    'previous_discount_weight',
    # 'item_cnt', 
    # 'pos_weighted_cnt',
    # 'avg_pos', 'median_pos'
]

categorical_feature=['locale']

# Function

## More Features 

In [80]:
session_features = pl.scan_parquet('../data/session_item_features/session_features.parquet')
item_features = pl.scan_parquet('../data/session_item_features/item_features.parquet')
product_pl = (pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 
                                          'products_train.parquet'))
                  # .with_columns(
                  #     pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                  #       .then(2)
                  #       .otherwise(3).alias('locale')
                  # )
             )

def get_additional_feature(data_pl, session_features=session_features, product_pl=product_pl, item_features=item_features):
    session_join_keys = ['session_id']
    session_fea_cols = ['mean_price', 'last_price', 'min_price', 'max_price']
    data_pl = (
        data_pl
         .with_columns(
            pl.col('session_id').cast(pl.Int64)
        )
        .join(session_features.select(session_join_keys+session_fea_cols+['locale']),
                                    how='left', on=session_join_keys, suffix='_str')
            .join(item_features, how='left',
                  left_on='next_item_prediction',
                  right_on='prev_items')
            .join(product_pl.select(['id', 'locale', 'price']), how='left', right_on=['id', 'locale']
                 , left_on=['next_item_prediction', 'locale_str'])
            .with_columns(
              pl.col('price').truediv(pl.col('mean_price')).alias('price_mean_norm')
              , pl.col('price').truediv(pl.col('last_price')).alias('price_last_norm')
              , pl.col('price').truediv(pl.col('min_price')).alias('price_min_norm')
              , pl.col('price').truediv(pl.col('max_price')).alias('price_max_norm')
              # , pl.when(pl.col('min_price')==pl.col('max_price')).then(1).otherwise((pl.col('price')-pl.col('min_price'))/(pl.col('min_price')-pl.col('max_price'))).alias('price_norm')
            )
            .select(
                pl.all().exclude(session_fea_cols+['locale_str', 'price'])
            )

    )
    return data_pl

# Read data

In [81]:

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, f'sessions_test_{task}.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))
test4task3_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))



In [82]:
! ls {ranker_train_data_dir}

eval.parquet  test.parquet  test4task3.parquet	train


In [83]:
ranker_train_data_dir

'../data/rank_train_data_task2_v2'

In [84]:
train_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train/' '*.parquet'), n_rows=SAMPLE_NUM)
# train_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train/' 'part_5.parquet'), n_rows=SAMPLE_NUM)

eval_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'), n_rows=SAMPLE_NUM)


In [85]:
test_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'), n_rows=SAMPLE_NUM)
test4task3_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test4task3.parquet'), n_rows=SAMPLE_NUM)

In [86]:
# train_candidates.collect().shape

In [87]:
! ls {ranker_train_data_dir}/train

part_1.parquet	 part_14.parquet  part_19.parquet  part_5.parquet
part_10.parquet  part_15.parquet  part_2.parquet   part_6.parquet
part_11.parquet  part_16.parquet  part_20.parquet  part_7.parquet
part_12.parquet  part_17.parquet  part_3.parquet   part_8.parquet
part_13.parquet  part_18.parquet  part_4.parquet   part_9.parquet


In [88]:
dic = eval_candidates.schema

In [89]:
dic.keys()

dict_keys(['session_id', 'next_item_prediction', 'w2v_weight', 'next_item', 'locale', 'prev_length', 'target', 'next_item_weight', 'next_item_cnt', 'next_item_weight_nfi', 'next_item_normalized_weight', 'next_item_weight_co_visit', 'discount_weight', 'following_weight', 'following_discount_weight', 'previous_weight', 'previous_discount_weight', 'last_item_similarity'])

In [90]:
train_candidates.schema

{'session_id': Float64,
 'next_item': Utf8,
 'next_item_prediction': Utf8,
 'target': Float64,
 'w2v_weight': Float64,
 'locale': Float64,
 'prev_length': Float64,
 'next_item_weight': Float64,
 'next_item_cnt': Float64,
 'next_item_weight_nfi': Float32,
 'next_item_normalized_weight': Float64,
 'next_item_weight_co_visit': Float32,
 'discount_weight': Float32,
 'following_weight': Float32,
 'following_discount_weight': Float32,
 'previous_weight': Float32,
 'previous_discount_weight': Float32,
 'last_item_similarity': Float32}

# Ranker 

## Train model

In [91]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    n_estimators=estimator, 
    importance_type='gain',
    eval_at=[3]
)

### eval data for training 

In [92]:
# eval_candidates.head().collect()

In [93]:
# train_candidates.collect().shape

In [94]:
# test_candidates.head(3).collect()

In [95]:
train_data_source = 'train' # train or 'eval'

if train_data_source == 'eval':
    target_df = eval_candidates
    negative_sampling = 0.1

    cols = ['session_id', 'next_item', 'next_item_prediction', 'target']
    sample_pl = target_df.select(cols)
    pos = sample_pl.filter(pl.col('target')==1)
    neg = (sample_pl
               .filter(pl.col('target')==0)
          )
    neg_original_len = neg.select('session_id').collect().shape[0]
    print(f'Original negative num: {neg_original_len}')
    neg = neg.groupby('session_id').agg(
        pl.all().sample(frac=negative_sampling)
        )
    neg = neg.explode(neg.columns[1:])
    # # print(neg.shape)
    # # print(neg.head().collect())
    sample_pl = pl.concat([pos, neg], how='vertical')
    target_df = sample_pl.join(target_df, how='left', on=cols).collect().sort('session_id').lazy()
    del sample_pl
else:
    target_df = train_candidates
    del train_candidates

### Train 

In [96]:
# target_df.schema

In [97]:
target_train_candidates = target_df
del target_df

In [98]:
# target_train_candidates.head().collect()

In [99]:
# session_features.head().collect()

In [100]:
# product_pl.head().collect()

In [101]:
# product_pl.head().collect()

In [102]:
# session_features.schema

In [103]:
# target_train_candidates

In [104]:
# data_pl = target_train_candidates

In [105]:
# data_pl.select('locale').collect()

In [106]:
# data_pl.schema

In [107]:
# session_features.schema

In [108]:
# session_join_keys = ['session_id']
# session_fea_cols = ['mean_price', 'last_price', 'min_price', 'max_price']
# # data_pl = 
# (
#     data_pl
#     .with_columns(
#         pl.col('session_id').cast(pl.Int64)
#     )
#     .join(session_features.select(session_join_keys+session_fea_cols+['locale']),
#                                 how='left', on=session_join_keys, suffix='_str')
#         .join(product_pl.select(['id', 'locale', 'price']), how='left', right_on=['id', 'locale']
#              , left_on=['next_item_prediction', 'locale_str'])
#         .with_columns(
#           pl.col('price').truediv(pl.col('mean_price')).alias('price_mean_norm')
#           , pl.col('price').truediv(pl.col('last_price')).alias('price_last_norm')
#           , pl.col('price').truediv(pl.col('min_price')).alias('price_min_norm')
#           , pl.col('price').truediv(pl.col('max_price')).alias('price_max_norm')
#           # , pl.when(pl.col('min_price')==pl.col('max_price')).then(1).otherwise((pl.col('price')-pl.col('min_price'))/(pl.col('min_price')-pl.col('max_price'))).alias('price_norm')
#         )
#         .select(
#             pl.all().exclude(session_fea_cols+['locale_str', 'price'])
#         )

# ).head().collect()#.select(['session_id','next_item_prediction', 'locale_str'])

In [109]:
# session_features.collect().shape

In [110]:
target_train_candidates = get_additional_feature(data_pl=target_train_candidates)

In [111]:
target_train_candidates.schema.keys()

dict_keys(['session_id', 'next_item', 'next_item_prediction', 'target', 'w2v_weight', 'locale', 'prev_length', 'next_item_weight', 'next_item_cnt', 'next_item_weight_nfi', 'next_item_normalized_weight', 'next_item_weight_co_visit', 'discount_weight', 'following_weight', 'following_discount_weight', 'previous_weight', 'previous_discount_weight', 'last_item_similarity', 'item_cnt', 'pos_weighted_cnt', 'avg_pos', 'median_pos', 'price_mean_norm', 'price_last_norm', 'price_min_norm', 'price_max_norm'])

In [112]:
train_session_id = target_train_candidates.select(pl.col('session_id')).unique().collect()

session_num = train_session_id.shape[0]

In [113]:
random_pl = train_session_id.with_columns(
        pl.Series(name='random', values=np.random.uniform(size=session_num))
)

In [114]:
train_candidates = target_train_candidates.join(random_pl.lazy(), how='left', on='session_id')

sampled_train_candidates = train_candidates.filter(pl.col('random')>eval_frac).sort('session_id', descending=False).collect()
sampled_eval_candidates = train_candidates.filter(pl.col('random')<=eval_frac).sort('session_id', descending=False).collect()

In [115]:
print('Train')
print(sampled_train_candidates.shape)
print(sampled_train_candidates.select('target').mean())
print()
print()
print('Eval')
print(sampled_eval_candidates.shape)
print(sampled_eval_candidates.select('target').mean())

Train
(1424875, 27)
shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.065527 │
└──────────┘


Eval
(157600, 27)
shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.065241 │
└──────────┘


In [116]:
# sampled_train_candidates.head()

<!-- no positive items ->  -->

In [117]:
# sampled_train_candidates.filter(pl.col('session_id')==0).sort('target', descending=True)

In [118]:
# sampled_train_candidates.filter(pl.col('session_id')==5).sort('target', descending=True)

In [119]:
# sampled_train_candidates.head()

In [120]:
# sampled_train_candidates.groupby('session_id').agg(pl.count())

In [121]:
session_lengths_train = sampled_train_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()
session_lengths_eval = sampled_eval_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()

In [122]:
pd.DataFrame(session_lengths_train).describe([0.05*i for i in range(20)])

,0
count,93368.000000
mean,15.260850
std,6.748718
min,2.000000
0%,2.000000
5%,10.000000
10%,11.000000
15%,11.000000
20%,11.000000
25%,11.000000


In [123]:
pd.DataFrame(session_lengths_eval).describe([0.05*i for i in range(20)])

,0
count,10282.000000
mean,15.327757
std,6.777262
min,2.000000
0%,2.000000
5%,10.000000
10%,11.000000
15%,11.000000
20%,11.000000
25%,11.000000


In [124]:
ranker = ranker.fit(
    X=sampled_train_candidates[feature_cols].to_pandas(),
    y=sampled_train_candidates[target].to_pandas(),
    group=session_lengths_train,
    eval_set=[(sampled_train_candidates[feature_cols].to_pandas(), sampled_train_candidates[target].to_pandas()),
           (sampled_eval_candidates[feature_cols].to_pandas(), sampled_eval_candidates[target].to_pandas())
             ],
    eval_group=[session_lengths_train,
                session_lengths_eval
               ]
    , categorical_feature=categorical_feature
    , early_stopping_rounds=early_stop_rounds
    # , callbacks=[early_stopping(early_stop_rounds)]
)

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:621: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib

[1]	valid_0's ndcg@3: 0.848282	valid_1's ndcg@3: 0.850905
[2]	valid_0's ndcg@3: 0.834972	valid_1's ndcg@3: 0.83783
[3]	valid_0's ndcg@3: 0.830569	valid_1's ndcg@3: 0.833816
[4]	valid_0's ndcg@3: 0.829296	valid_1's ndcg@3: 0.832866
[5]	valid_0's ndcg@3: 0.828288	valid_1's ndcg@3: 0.830956
[6]	valid_0's ndcg@3: 0.827915	valid_1's ndcg@3: 0.830247
[7]	valid_0's ndcg@3: 0.827939	valid_1's ndcg@3: 0.829559
[8]	valid_0's ndcg@3: 0.828007	valid_1's ndcg@3: 0.829761
[9]	valid_0's ndcg@3: 0.827989	valid_1's ndcg@3: 0.830624
[10]	valid_0's ndcg@3: 0.828309	valid_1's ndcg@3: 0.830498
[11]	valid_0's ndcg@3: 0.828414	valid_1's ndcg@3: 0.830377


In [125]:
print(sampled_train_candidates.estimated_size(unit='gb'))
del sampled_train_candidates

0.2667052894830704


In [126]:
print(sampled_eval_candidates.estimated_size(unit='gb'))
del sampled_eval_candidates

0.029499399475753307


## Save model 

In [127]:
debug

False

In [128]:
rank_model_dir

'../model_training/task2_rank_lgbm_v2'

In [129]:
if not debug:
    joblib.dump(
            value=ranker,
            filename=os.path.join(rank_model_dir, 'model.pkl')
    )

In [130]:
if not debug:
    del ranker

## Load Model 

In [131]:
debug

False

In [132]:
if not debug:
    ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))
ranker

LGBMRanker(eval_at=[3], importance_type='gain', metric='ndcg',
           n_estimators=1000, objective='lambdarank')

### Importance 

In [133]:
impotant_df = pd.DataFrame(
    {
        'features': ranker.feature_name_,
        'importance': ranker.feature_importances_
    }
).sort_values('importance', ascending=False)
impotant_df

,features,importance
12,next_item_normalized_weight,277530.813019
7,price_mean_norm,63774.749939
5,last_item_similarity,10895.137848
13,discount_weight,10731.409607
17,previous_discount_weight,10358.424759
10,price_max_norm,2173.659912
4,w2v_weight,1722.880005
9,price_min_norm,1054.976959
8,price_last_norm,490.808990
11,next_item_cnt,361.186005


# Eval ranker 

In [134]:
def rank_rec(candidate_pl, feature_cols, ranker, topn=100):

    candidate_pl = candidate_pl.collect()
    print(f"candidate pl shape: {candidate_pl.shape}")
    inference = ranker.predict(candidate_pl[feature_cols].to_pandas())
    test_result = (candidate_pl
         .lazy()
         .with_columns(
             pl.Series(name='predict', values=inference)
         )
         .with_columns(
            pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
         )
         .sort(['session_id', 'rank'])
         .filter(pl.col('rank')<=topn)
         .groupby(['session_id'])
         .agg(
             pl.col('next_item_prediction')
             , pl.col('next_item').unique()#.arr.get(0).cast(pl.Utf8)
         ).with_columns(
             pl.col('next_item').arr.get(0).alias('next_item')
         )
    )
    return test_result


In [135]:
# target_train_candidates.head().collect()

In [136]:
eval_candidates = get_additional_feature(data_pl=eval_candidates)

In [137]:
ranker_eval_pl = rank_rec(candidate_pl=eval_candidates,
                         feature_cols=feature_cols
                         , ranker=ranker
                        # , topn=10
                         )

candidate pl shape: (4660340, 26)


In [138]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
33333,0.3446,0.5993,0.6843


In [292]:
# model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

# Test4task3 

In [293]:
topn=1
test4task3_candidates = get_additional_feature(data_pl=test4task3_candidates)
test4task3_candidates = test4task3_candidates.collect()
inference = ranker.predict(test4task3_candidates[feature_cols].to_pandas())
test4task3_result = (test4task3_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=topn)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test4task3_result.head().collect()
predictions = test4task3_pl.join(test4task3_result, how='left', on='session_id').collect()
# [['locale', 'next_item_prediction']].to_pandas()

In [294]:
test4task3_sub_file = f'../data/sub_files/{task}_test4task3_{rank_model_version}.parquet'
print(test4task3_sub_file)
predictions.write_parquet(test4task3_sub_file)
predictions.select('locale').to_series().value_counts()

../data/sub_files/task2_test4task3_task2_rank_lgbm_v2.parquet


locale,counts
str,u32
"""FR""",10000
"""IT""",10000
"""ES""",6422


# Test inference 

In [295]:
test_candidates = get_additional_feature(data_pl=test_candidates)
test_candidates = test_candidates.collect()
inference = ranker.predict(test_candidates[feature_cols].to_pandas())
test_result = (test_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=100)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test_result.head().collect()
predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [296]:
predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [297]:
predictions.shape

(34690, 2)

In [298]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend
if not debug:
    predictions.to_parquet(sub_file, engine='pyarrow')

In [299]:
sub_file

'../data/sub_files/task2_rank_lgbm_v2.parque'

# Submit result 

In [88]:
# # You can submit with aicrowd-cli, or upload manually on the challenge page.
# !aicrowd submission create -c task-1-next-product-recommendation -f {sub_file}

#  Case Study from taks2

In [92]:
product_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 
                                          'products_train.parquet'))

In [93]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
33333,0.3242,0.5594,0.6419


In [94]:
target_df = eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id')

In [131]:
# target_df.head().collect()

In [96]:
eval_final = (
        target_df
        .lazy()
        .with_columns(
            pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
        )
        .with_columns(
            pl.concat_list([pl.col('next_item'), pl.col('next_item_prediction')]).alias('mrr')
        )
        .with_columns(
            pl.col('mrr').arr.eval(
                pl.arg_where(pl.element()==pl.element().first())
            )
        ).with_columns(
            pl.col('mrr').arr.eval(
                pl.when(pl.element()==0).then(0).otherwise(1/pl.element())
            )
        ).with_columns(
            pl.col('mrr').arr.sum()
            , pl.col('next_item_prediction').arr.head(1).arr.contains(pl.col('next_item')).alias('recall@1')
            , pl.col('next_item_prediction').arr.head(3).arr.contains(pl.col('next_item')).alias('recall@3')
            , pl.col('next_item_prediction').arr.head(20).arr.contains(pl.col('next_item')).alias('recall@20')
            , pl.col('next_item_prediction').arr.head(100).arr.contains(pl.col('next_item')).alias('recall@100')

        )
)
final_res = eval_final.select(
        pl.count().alias('total_sessions')
        , pl.col('mrr').mean().round(4)
        , pl.col('recall@1').mean().round(4)
        , pl.col('recall@3').mean().round(4)
        , pl.col('recall@20').mean().round(4)
        , pl.col('recall@100').mean().round(4)

    ).collect()
final_res

total_sessions,mrr,recall@1,recall@3,recall@20,recall@100
u32,f64,f64,f64,f64,f64
33333,0.3242,0.237,0.3675,0.5594,0.6419


In [98]:
# final_res

In [99]:
# eval_final.filter(pl.col('recall@1')==True).head().collect()

In [100]:
# eval_final.head().collect()

## recall@3 = True and recall@1=False 

In [101]:
all_cases = eval_final.filter(pl.col('recall@1')==False).filter(pl.col('recall@3')==True)

In [102]:
all_cases = (
    all_cases.with_columns(
        pl.col('next_item_prediction').arr.head(3).alias('next_item_prediction')
    )
)

In [150]:
all_cases.collect().sample(5)

prev_items,next_item,locale,session_id,next_item_prediction,next_item_right,mrr,recall@1,recall@3,recall@20,recall@100
list[str],str,str,i64,list[str],str,f64,bool,bool,bool,bool
"[""B00NPXEEC4"", ""B07JN5C1S3"", … ""B0B3JS3NFN""]","""B09NQHCKJK""","""IT""",3575764,"[""B08P4YPB8Q"", ""B0092QK8N2"", ""B09NQHCKJK""]","""B09NQHCKJK""",0.333333,false,true,true,true
"[""B06Y2H5JGX"", ""B0811QN9PN""]","""B0B1J56P98""","""FR""",3407839,"[""B09ZXWHPRD"", ""B0B1J56P98"", ""B07GX1WYFC""]","""B0B1J56P98""",0.5,false,true,true,true
"[""B08BYJ9Q65"", ""B08HRG6W11"", ""B08BYJ9Q65""]","""B08TLYM129""","""IT""",3537237,"[""B09G5N3D36"", ""B08TLYM129"", ""B086XFFD5Y""]","""B08TLYM129""",0.5,false,true,true,true
"[""B01N1NR31G"", ""B01MZ5ELRT""]","""B01F549LFK""","""IT""",3538047,"[""B01N5L1J0P"", ""B01F549LFK"", ""B01MSUK566""]","""B01F549LFK""",0.5,false,true,true,true
"[""B003Z0HU0K"", ""B003Z0HU0K"", ""B09NFV8PBV""]","""B09NGFB7KD""","""FR""",3367530,"[""B09NFT49PW"", ""B09NGFB7KD"", ""2226328432""]","""B09NGFB7KD""",0.5,false,true,true,true


## Function 

In [143]:
# def print_single_case(session_id, all_cases):
#     feature = (
#      all_cases
#         .select(['next_item_prediction'])
#         .filter(pl.col('session_id')==session_id).explode('next_item_prediction')
#         .join(
#             eval_candidates.filter(pl.col('session_id')==session_id)
#             , how='left'
#             , on='next_item_prediction'
#         )
# )  
#     print()
#     print("features")
#     print(feature.collect())
#     print()
#     print()
#                   # e
    

### 3272776 --> hard to locate problem; `next_item_weight_co_visit` 

In [132]:
session_id = 3272776

In [142]:
# print_single_case(session_id=session_id, all_cases=all_cases)

In [133]:
single_case = all_cases.filter(pl.col('session_id')==session_id).collect()
single_case

prev_items,next_item,locale,session_id,next_item_prediction,next_item_right,mrr,recall@1,recall@3,recall@20,recall@100
list[str],str,str,i64,list[str],str,f64,bool,bool,bool,bool
"[""B0B2K87MXS"", ""B0B2KBH114"", … ""B0B2K87MXS""]","""B0B4612MTM""","""ES""",3272776,"[""B09PNDJJZ9"", ""B08FMPXDTJ"", ""B0B4612MTM""]","""B0B4612MTM""",0.333333,false,true,true,true


In [134]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl)

++++++++++++++++++++
0: B0B2K87MXS
++++++++++++++++++++
===title====
    XIAOMI Xiaomi Smartphone redmi 9c NFC 3/64gb Verde Libre

===desc====
    Alta calidad

unique_id                               1416732
price                                    116.76
brand                                    Xiaomi
color                              Aurora Green
size         Redmi 9C NFC 3GB+64GB Aurora Green
model                     REDMI 9C NFC 3-64 GRE
material                                   None
author                                     None
Name: 0, dtype: object


++++++++++++++++++++
1: B0B2K87MXS
++++++++++++++++++++
===title====
    XIAOMI Xiaomi Smartphone redmi 9c NFC 3/64gb Verde Libre

===desc====
    Alta calidad

unique_id                               1416732
price                                    116.76
brand                                    Xiaomi
color                              Aurora Green
size         Redmi 9C NFC 3GB+64GB Aurora Green
model                     RED

In [135]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl
                , item_list_col='next_item_prediction')

++++++++++++++++++++
0: B08FMPXDTJ
++++++++++++++++++++
===title====
    Xiaomi Redmi 9C NFC-Smartphone de 6.53" HD+ (3GB+64GB, 3x cámara trasera de 13MP con IA, MediaTek Helio G35, Batería 5000 mAh, 10 W carga rápida), Gris

===desc====
    Batería de alta capacidad de 5000 mah con carga rápida de 10w.

unique_id          1429975
price                121.0
brand               Xiaomi
color        Midnight Gray
size            3GB + 64GB
model        6941059651307
material              None
author              Xiaomi
Name: 0, dtype: object


++++++++++++++++++++
1: B09PNDJJZ9
++++++++++++++++++++
===title====
    Xiaomi Redmi 9C - Smartphone 128GB, 4GB RAM, Dual Sim, Aurora Green

===desc====
    Batería: 5000 mAh

unique_id                            1430860
price                                  142.0
brand                                 Xiaomi
color                           Aurora Green
size         Redmi 9C 4GB+128GB Aurora Green
model                              MZB0A46EU
materi

In [136]:
feature = (
     all_cases
        .select(['next_item_prediction'])
        .filter(pl.col('session_id')==session_id).explode('next_item_prediction')
        .join(
            eval_candidates.filter(pl.col('session_id')==session_id)
            , how='left'
            , on='next_item_prediction'
        )
)  

feature.collect()
                  # e

next_item_prediction,session_id,w2v_weight,next_item,locale,prev_length,target,next_item_weight,next_item_weight_nfi,next_item_weight_co_visit,last_item_similarity,price_mean_norm,price_last_norm,price_min_norm,price_max_norm
str,i64,f64,str,i32,u32,i32,f64,f32,f32,f32,f64,f64,f64,f64
"""B09PNDJJZ9""",3272776,0.97,"""B0B4612MTM""",3,4,0,1.01,5.83,12.0,0.992541,1.321176,1.21617,1.449127,1.21617
"""B08FMPXDTJ""",3272776,0.995,"""B0B4612MTM""",3,4,0,0.26,2.25,8.0,0.994898,1.125791,1.036314,1.23482,1.036314
"""B0B4612MTM""",3272776,0.81,"""B0B4612MTM""",3,4,1,0.26,3.95,11.0,0.97156,1.292799,1.190048,1.418002,1.190048


### 3373971 -> reply on NIC too much; `w2v_weight` -> not zero for ground true 

In [145]:
session_id = 3373971

In [146]:
feature = (
     all_cases
        .select(['next_item_prediction'])
        .filter(pl.col('session_id')==session_id).explode('next_item_prediction')
        .join(
            eval_candidates.filter(pl.col('session_id')==session_id)
            , how='left'
            , on='next_item_prediction'
        )
)  

feature.collect()
                  # e

next_item_prediction,session_id,w2v_weight,next_item,locale,prev_length,target,next_item_weight,next_item_weight_nfi,next_item_weight_co_visit,last_item_similarity,price_mean_norm,price_last_norm,price_min_norm,price_max_norm
str,i64,f64,str,i32,u32,i32,f64,f32,f32,f32,f64,f64,f64,f64
"""B015TJ8NM8""",3373971,0.0,"""B000GIPJ02""",3,5,0,0.76,5.16,15.0,0.959601,0.772395,1.0,1.0,0.564819
"""B000GIPJ02""",3373971,0.89,"""B000GIPJ02""",3,5,1,0.51,3.0,5.0,0.853157,1.315306,1.702893,1.702893,0.961826
"""B002DYIZPS""",3373971,0.0,"""B000GIPJ02""",3,5,0,0.0,0.5,1.0,0.793911,1.367509,1.770479,1.770479,1.0


In [147]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl)

++++++++++++++++++++
0: B000GIPJ0M
++++++++++++++++++++
===title====
    Optimum Nutrition Serious Mass, Mass Gainer avec Whey, Proteines Musculation Prise de Masse avec Vitamines, Creatine et Glutamine, Chocolat, 16 Portions, 5.45kg, l'Emballage Peut Varier

===desc====
    AUGMENTATEUR DE MASSE MUSCULAIRE avec un nombre exorbitant de 1250 calories et 50 g de whey protéines par portion. Contient moins de 5 g de graisse !

unique_id               1469992
price                     62.99
brand         Optimum Nutrition
color                  Chocolat
size         5.45 kg (Lot de 1)
model                   1101529
material              Palstique
author                     None
Name: 0, dtype: object


++++++++++++++++++++
1: B09RMWFZ8W
++++++++++++++++++++
===title====
    Optimum Nutrition | Serious Mass (2,73kg) | Gainers | Puissant accélérateur de masse

===desc====
    Augmentation de la prise de masse

unique_id               1470603
price                     36.99
brand         Opti

In [148]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl
                , item_list_col='next_item_prediction')

++++++++++++++++++++
0: B015TJ8NM8
++++++++++++++++++++
===title====
    Optimum Nutrition Serious Mass, Mass Gainer avec Whey, Proteines Musculation Prise de Masse avec Vitamines, Creatine et Glutamine, Chocolat Beurre de Cacahuète, 8 Portions, l'Emballage Peut Varier

===desc====
    AUGMENTATEUR DE MASSE MUSCULAIRE avec un nombre incroyable de 1250 calories et 50 g de whey protéines par portion. Contient moins de 5 g de graisse

unique_id                          1459015
price                                36.99
brand        Glanbia Performance Nutrition
color         Chocolat Beurre de Cacahuète
size                    2.73 kg (Lot de 1)
model                              1101541
material                              None
author                                None
Name: 0, dtype: object


++++++++++++++++++++
1: B000GIPJ02
++++++++++++++++++++
===title====
    Optimum Nutrition Serious Mass, Mass Gainer avec Whey, Proteines Musculation Prise de Masse avec Vitamines, Creatine et Gl

### 3407839 w2v_weight is 0??;  看不出来？？？

In [152]:
session_id = 3407839

In [153]:
feature = (
     all_cases
        .select(['next_item_prediction'])
        .filter(pl.col('session_id')==session_id).explode('next_item_prediction')
        .join(
            eval_candidates.filter(pl.col('session_id')==session_id)
            , how='left'
            , on='next_item_prediction'
        )
)  

feature.collect()
                  # e

next_item_prediction,session_id,w2v_weight,next_item,locale,prev_length,target,next_item_weight,next_item_weight_nfi,next_item_weight_co_visit,last_item_similarity,price_mean_norm,price_last_norm,price_min_norm,price_max_norm
str,i64,f64,str,i32,u32,i32,f64,f32,f32,f32,f64,f64,f64,f64
"""B09ZXWHPRD""",3407839,0.0,"""B0B1J56P98""",3,2,0,1.01,1.0,1.0,0.951582,1.258554,1.054069,1.561474,1.054069
"""B0B1J56P98""",3407839,0.0,"""B0B1J56P98""",3,2,1,1.01,1.0,1.0,0.943084,1.290833,1.081103,1.601522,1.081103
"""B07GX1WYFC""",3407839,0.96,"""B0B1J56P98""",3,2,0,0.0,0.0,0.0,0.963218,0.659458,0.552311,0.818182,0.552311


In [154]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl)

++++++++++++++++++++
0: B06Y2H5JGX
++++++++++++++++++++
===title====
    ANSIO Fer à repasser à vapeur en céramique, 2000 W, cordon d'alimentation 3m / 10ft, réservoir d'eau 200ml - violet/blanc

===desc====
    CONTRÔLE DE TEMPÉRATURE RÉGLABLE: Le réglage de température précis permet de faire correspondre le type de vêtement pour une finition parfaite.

unique_id      1489916
price            24.97
brand            ANSIO
color           Violet
size              None
model             None
material     Céramique
author            None
Name: 0, dtype: object


++++++++++++++++++++
1: B0811QN9PN
++++++++++++++++++++
===title====
    Yabano Fer a Repasser, Fer à Vapeur 2400W, 350 ml Réservoir, Fonction Pressing 320g/min, Semelle en Céramique, 2,5m Cable, Auto-nettoyage

===desc====
    💨2400 Watts, Semelle céramique à trous multiples.

unique_id                1491923
price                      36.99
brand                     Yabano
color                        Blu
size                   

In [155]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl
                , item_list_col='next_item_prediction')


++++++++++++++++++++
0: B0B1J56P98
++++++++++++++++++++
===title====
    Cozeemax Fer à vapeur 2400 W, 320 g/min - Semelle en céramique - Nettoyage automatique et explosion - Arrêt de goutte - Réservoir d'eau de 300 ml - Vapeur verticale

===desc====
    Thermostat réglable et vapeur : un contrôle précis de la température et un contrôle de vapeur variable peuvent protéger vos vêtements dans les meilleures conditions. Parfait pour tous les tissus tels que le nylon, la soie, la laine, le coton, les jeans et le lin.

unique_id     1475018
price           39.99
brand        Cozeemax
color            None
size             None
model          AJ2016
material         None
author           None
Name: 0, dtype: object


++++++++++++++++++++
1: B09ZXWHPRD
++++++++++++++++++++
===title====
    Défroisseur vapeur vertical portable voyage, 280ml, 1500W Fer à vapeur pratique Chauffage rapide, vapeur 30g/min, pour vêtements, rideaux, meubles rembourrés

===desc====
    【Conception de sécurité】 Chaque

## recall@100=False 